Gewünschte Lösung siehe unten - fehlende Unterstützung von Wordnet für deutsche Sprache. Abweichende Implementierung mit openthesaurus

In [ ]:
import pandas as pd
from tqdm import tqdm
import spacy
from nltk.corpus import wordnet
import nltk

# Sicherstellen, dass alle erforderlichen NLTK-Resourcen geladen sind
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Laden des SpaCy-Modells
nlp = spacy.load("de_core_news_sm")  # Deutsches Modell verwenden, falls der Text deutsch ist

file_path = "v2_1.csv"
df = pd.read_csv(file_path)

normalized_texts = []

def get_synonym(word):
    """Findet ein Synonym für ein gegebenes Wort mit WordNet."""
    synonyms = wordnet.synsets(word, lang='de')  # 'lang' für Deutsch anpassen
    if synonyms:
        # Nimmt das erste Synonym
        return synonyms[0].lemmas(lang='de')[0].name()
    return word

for text in tqdm(df['text'], desc="Verarbeitung der Reden"):
    tokens = text.split()  # Annahme: Der Text ist bereits vorverarbeitet und tokenisiert
    synonym_replaced_tokens = [
        get_synonym(token) for token in tokens
    ]
    normalized_texts.append(" ".join(synonym_replaced_tokens))

# Die 'text'-Spalte mit den normalisierten Texten überschreiben
df['text'] = normalized_texts

output_path = "v2_2.csv"
df.to_csv(output_path, index=False)
print(f"Normalisierte Daten mit Synonymen gespeichert unter: {output_path}")

Schwierige Formattierung -- txt nicht wirklich verwertbar - Verusch der Synonym entfernung durch sql

In [6]:
import pandas as pd
from tqdm import tqdm
import spacy
import re

# Laden des SpaCy-Modells
nlp = spacy.load("de_core_news_sm")  # Deutsches Modell verwenden

file_path = "v2_1.csv"
df = pd.read_csv(file_path)

# Funktion zum Parsen der OpenThesaurus-Daten
def load_synonym_dictionary(file_path):
    """Parst die OpenThesaurus-Datenbank und erstellt ein Synonym-Wörterbuch."""
    synonym_dict = {}
    removed_synonyms_count = 0
    with open(file_path, encoding="utf-8") as file:
        for line in file:
            if line.strip() and not line.startswith("#"):
                # Entfernt alles, was in Klammern steht
                removed_synonyms_count += len(re.findall(r'\([^)]*\)', line))
                line = re.sub(r'\([^)]*\)', '', line)
                # Entfernt "..." aus der Zeile
                line = re.sub(r'\.\.\.', '', line)
                words = [w.strip() for w in line.split(";")]
                
                # Priorisierung: Hauptform an den Anfang setzen
                main_form = None
                for word in words:
                    if "Hauptform" in word:  # Erkennen der Hauptform
                        main_form = word.replace("Hauptform", "").strip()
                        break
                if main_form:
                    # Hauptform an den Anfang der Liste setzen
                    words = [main_form] + [w for w in words if w != main_form]
                
                for word in words:
                    synonym_dict[word] = [w.strip() for w in words]  # Synonyme bereinigen
    print(f"Anzahl der entfernten Synonyme: {removed_synonyms_count}")
    return synonym_dict

# Pfad zur heruntergeladenen OpenThesaurus-Datei
thesaurus_path = "openthesaurus.txt"
synonym_dict = load_synonym_dictionary(thesaurus_path)

# Funktion zur Synonym-Ersetzung
def get_synonym(word):
    """Findet ein Synonym aus dem Synonym-Wörterbuch."""
    synonyms = synonym_dict.get(word, [word])  # Falls kein Synonym vorhanden, Rückgabe des Wortes selbst
    return synonyms[0]  # Erster Eintrag als Ersatz

normalized_texts = []
total_replacements = 0  # Zähler für ersetzte Wörter

for text in tqdm(df['text'], desc="Verarbeitung der Reden"):
    tokens = text.split()  # Annahme: Der Text ist bereits vorverarbeitet und tokenisiert
    synonym_replaced_tokens = []
    for token in tokens:
        clean_token = re.sub(r'[\W_]', '', token)  # Entfernt Sonderzeichen aus jedem Token
        if clean_token:  # Nur nicht-leere Tokens weiterverarbeiten
            replacement = get_synonym(clean_token)
            if replacement != clean_token:
                total_replacements += 1
            synonym_replaced_tokens.append(replacement)
        else:
            synonym_replaced_tokens.append(token)  # Belässt ursprüngliches Token bei Fehlern
    normalized_texts.append(" ".join(synonym_replaced_tokens))

# Ausgabe der Anzahl der ersetzten Wörter
print(f"Anzahl der ersetzten Wörter: {total_replacements}")

# Die 'text'-Spalte mit den normalisierten Texten überschreiben
df['text'] = normalized_texts

output_path = "v2_2_2_synonym.csv"
df.to_csv(output_path, index=False)
print(f"Normalisierte Daten mit Synonymen gespeichert unter: {output_path}")


Anzahl der entfernten Synonyme: 85692


Verarbeitung der Reden: 100%|██████████| 34428/34428 [00:18<00:00, 1880.50it/s]


Anzahl der ersetzten Wörter: 8934148
Normalisierte Daten mit Synonymen gespeichert unter: v2_2_2_synonym.csv


In [9]:
import pandas as pd
from tqdm import tqdm
import spacy
import re
import mysql.connector

# Laden des SpaCy-Modells
nlp = spacy.load("de_core_news_sm")  # Deutsches Modell verwenden

# Verbindung zur MySQL-Datenbank herstellen
db_connection = mysql.connector.connect(
    host="localhost",       # MySQL-Host
    user="root",            # MySQL-Benutzername
    password="admin", # MySQL-Passwort
    database="openthesaurus" # Name der Datenbank
)
cursor = db_connection.cursor()

# Funktion zur Abfrage des bevorzugten Synonyms aus der Datenbank
def get_synonym_from_db(word):
    """Findet das bevorzugte Synonym für ein Wort in der MySQL-Datenbank."""
    query = """
    SELECT t2.word
    FROM term t1
    JOIN term t2 ON t1.synset_id = t2.synset_id
    JOIN synset s ON t1.synset_id = s.id
    WHERE t1.word = %s AND t2.word = s.synset_preferred_term;
    """
    cursor.execute(query, (word,))
    result = cursor.fetchone()
    return result[0] if result else word  # Rückgabe des bevorzugten Synonyms oder des Originalworts

file_path = "v2_1.csv"
df = pd.read_csv(file_path)

normalized_texts = []
total_replacements = 0  # Zähler für ersetzte Wörter

for text in tqdm(df['text'], desc="Verarbeitung der Reden"):
    tokens = text.split()  # Annahme: Der Text ist bereits vorverarbeitet und tokenisiert
    synonym_replaced_tokens = []
    for token in tokens:
        clean_token = re.sub(r'[\W_]', '', token)  # Entfernt Sonderzeichen aus jedem Token
        if clean_token:  # Nur nicht-leere Tokens weiterverarbeiten
            replacement = get_synonym_from_db(clean_token)
            if replacement != clean_token:
                total_replacements += 1
            synonym_replaced_tokens.append(replacement)
        else:
            synonym_replaced_tokens.append(token)  # Belässt ursprüngliches Token bei Fehlern
    normalized_texts.append(" ".join(synonym_replaced_tokens))

# Ausgabe der Anzahl der ersetzten Wörter
print(f"Anzahl der ersetzten Wörter: {total_replacements}")

# Die 'text'-Spalte mit den normalisierten Texten überschreiben
df['text'] = normalized_texts

output_path = "v2_2.csv"
df.to_csv(output_path, index=False)
print(f"Normalisierte Daten mit Synonymen gespeichert unter: {output_path}")

# Datenbankverbindung schließen
cursor.close()
db_connection.close()

Verarbeitung der Reden: 100%|██████████| 34428/34428 [58:21<00:00,  9.83it/s]  


Anzahl der ersetzten Wörter: 0
Normalisierte Daten mit Synonymen gespeichert unter: v2_2.csv


Vergleich der Datein - mit Synonymen und Ohne   

In [3]:
import pandas as pd
from collections import Counter
import string

# Hilfsfunktion zur Bereinigung von Text
def preprocess_text(text):
    text = text.lower()  # Umwandlung in Kleinbuchstaben
    text = text.translate(str.maketrans('', '', string.punctuation))  # Entfernen von Satzzeichen
    return text

# Funktion zur Analyse der Textunterschiede
def analyze_text_changes(file_original, file_cleaned, text_column):
    # CSV-Dateien laden
    df_original = pd.read_csv(file_original)
    df_cleaned = pd.read_csv(file_cleaned)

    # Textspalten extrahieren
    texts_original = df_original[text_column].dropna().apply(preprocess_text)
    texts_cleaned = df_cleaned[text_column].dropna().apply(preprocess_text)

    # Wortlisten erstellen
    words_original = Counter(" ".join(texts_original).split())
    words_cleaned = Counter(" ".join(texts_cleaned).split())

    # Einzigartige Wörter zählen
    unique_words_original = set(words_original.keys())
    unique_words_cleaned = set(words_cleaned.keys())

    # Vergleich der Wortanzahl
    total_words_original = sum(words_original.values())
    total_words_cleaned = sum(words_cleaned.values())

    # Synonyme (entfernte Wörter)
    removed_words = unique_words_original - unique_words_cleaned

    # Ergebnisse ausgeben
    results = {
        "Total Words (Original)": total_words_original,
        "Total Words (Cleaned)": total_words_cleaned,
        "Unique Words (Original)": len(unique_words_original),
        "Unique Words (Cleaned)": len(unique_words_cleaned),
        "Removed Synonyms (Count)": len(removed_words),
    }

    return results

# Dateien und Spaltennamen
file_original = "v2_2_stoppword_removal_wo_synonym_replacement.csv"
file_cleaned = "v2_2_stoppword_removal_w_synonym_replacement.csv"
text_column = "text"  # Ersetzen Sie dies mit dem Namen der Textspalte

# Analyse durchführen
results = analyze_text_changes(file_original, file_cleaned, text_column)

# Ergebnisse anzeigen
for key, value in results.items():
    print(f"{key}: {value}")

Total Words (Original): 7427891
Total Words (Cleaned): 7591942
Unique Words (Original): 219655
Unique Words (Cleaned): 228105
Removed Synonyms (Count): 14141


Results: Synonym Replacements

Total Words (Original): 18526649
Total Words (Cleaned): 18526649
Unique Words (Original): 288958
Unique Words (Cleaned): 268188
Removed Synonyms (Count): 24835

8,6% less unique entrys. 

Results - Stoppwordremoval, Lemmantizierung after Synonym Replacements
Total Words (Original): 7427891
Total Words (Cleaned): 7591942
Unique Words (Original): 219655
Unique Words (Cleaned): 228105

